In [41]:
from jupyter_plotly_dash import JupyterDash
import base64
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
# import method from python module
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
#FIXME: remove hard-coding of username/password
username = "aacuser"
password = "aac"
shelter = AnimalShelter(username, password)

# must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Rescue Aniimals')

# Grazioso Salvare’s logo
image_filename = './images/companylogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # header
    html.Div(id='hidden-div', style={'display':'flex'}),
    html.Div(
        html.A(
            href='https://www.snhu.com',
            #href='https://github.com/katiestapleton/PyMongoClientServer',
            children=[
                html.Img(id='companyLogo',
                    alt="Grazioso Salvare’s logo",
                    style = {
                        'Height' : '25%',
                        'maxHeight' : '150px',
                        'width': 'auto',
                        'padding' : '10px',
                    },
                    src='data:image/png;base64,{}'.format(encoded_image.decode())
                )
            ]
        )  ,
        style={
            'border': '5px solid #BF0404',
            'backgroundColor':'#F2F2F2',
            'textAlign' : 'center'
        },
    ),
    
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H3(html.Center("Hitchhiker's Guide to the Galaxy: Towel Search and Rescue Animals")),
    html.Hr(),
    html.Hr(),
    
    # body
    html.P("Review animals available for training. Select tab to filter data. Sort using the arrows in each column heading"),
    html.Div(
        dcc.Tabs(id='tabsFilter', value='tabDefault', 
        children=[
            dcc.Tab(label='All Animals', value='tabDefault'),
            dcc.Tab(label='Dogs', value='tabDogs'),
            dcc.Tab(label='Water', value='tabWater'),
            dcc.Tab(label='Wilderness', value='tabWilderness'),
            dcc.Tab(label='Tracking', value='tabTracking')            
        ],
        colors={
        "border": "#F2F2F2",
        "primary": "#BF0404",
        "background": "#D97373"
    }
        ),  
    ),
    html.Br(),
    
    # table to display animal data to user
    dt.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],        
        
        # style cells
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'minWidth': '100px',
            'maxWidth': '200px',
            'padding': '5px 10px',
            'textAlign': 'left'
        },
        style_header={
            'textAlign': 'center',
            'backgroundColor':'#F2F2F2',
            'fontWeight': 'bold'          
        },
                
        # edit permission
        editable=False,
                
        # filter & sort
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
      
        # page setup
        page_action="native",
        page_current= 0,
        page_size= 2
        
    ),
    html.Br(),
    html.Br(),
    html.Hr(),
    
    # chart and geolocation map are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
    ]),
    html.Hr(),
    html.Br(),
    
    # footer 
    html.P("\"The answer to the ultimate question of life, the universe and everything is 42.\" -H2G2."),
    html.P("Developed by Galaxy Travel Guide Katie Stapleton")
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    Output('datatable-id','data'),
    [Input("tabsFilter", "value")]
    )
def update_dashboard(tab):
    if tab == 'tabDefault':
        df_sub = pd.DataFrame.from_records(shelter.read({}))
    elif tab == 'tabDogs':
        df_sub = pd.DataFrame.from_records(shelter.read({"animal_type" : "Dog"}))   
    elif tab == 'tabWater':
        df_sub = pd.DataFrame.from_records(shelter.read({"$and":[
            {"breed" : {"$in":["Labrador Retriever Mix", "Chespeake Bay Retriever", 
                               "Newfoundland"]}},
            {"sex_upon_outcome" : "Intact Female"},
            {"age_upon_outcome_in_weeks" : {"$gt":26.0, "$lt":157.0}}
        ]}))
    elif tab == 'tabWilderness':
        df_sub = pd.DataFrame.from_records(shelter.read({"$and":[
            {"breed" : {"$in":["Doberman Pinscher", "German Shepherd","Golden Retriever", 
                               "Bloodhound", "Rottweiler"]}},
            {"sex_upon_outcome" : "Intact Male"},
            {"age_upon_outcome_in_weeks" : {"$gt":26.0, "$lt":157.0}}
        ]}))
    elif tab == 'tabTracking':
        df_sub = pd.DataFrame.from_records(shelter.read({"$and":[
            {"breed" : {"$in":["German Shepherd", "Alaskan Malamute", 
                               "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
            {"sex_upon_outcome" : "Intact Male"},
            {"age_upon_outcome_in_weeks" : {"$gt":20.0, "$lt":301.0}}
        ]}))
    df_update = df_sub.to_dict("records")
    return df_update


# datatable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    selection = [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
        } for i in selected_columns]
    return selection

#graph/chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #fig= [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
    fig = print("hello")
    return fig


# interactive geolocation map to display location of pets
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = [     
        dl.Map(style={'width': '1000px', 'height': '600px'}, center=[30.75,-97.48], zoom=10,      
        # slicing files/base source: https://community.plotly.com/t/trouble-with-dataframe-slicing-using-iloc-and-loc-in-dash/29410
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[(dff['location_lat'].iloc[1]),(dff['location_long'].iloc[1])],       
            children=[
                # hover ID
                dl.Tooltip(dff['animal_id'].iloc[1]),
                # info in hover
                dl.Popup([
                    html.H1('ID: ' + dff['animal_id'].iloc[1]),
                    html.P('Breed: ' + dff['breed'].iloc[1]),
                    html.P(dff['age_upon_outcome_in_weeks'].iloc[1]),
                    html.P('Name: ' + dff['name'].iloc[1])
                ])
            ])
        ])
    ]
    
    return fig
    
#!!do not remove!!
app